In [1]:
import pandas as pd
movies = pd.read_csv('data/movies.csv', sep=',')
movies.nunique()

movieId    9742
title      9737
genres      951
dtype: int64

In [2]:
rating1 = pd.read_csv('data/ratings1.csv', sep=',')
rating2 = pd.read_csv('data/ratings2.csv', sep=',')
rating1.nunique()

userId      274
movieId    6219
rating       10
dtype: int64

In [3]:
dates = pd.read_csv('data/dates.csv', sep=',')
dates.head()

,date
0,2000-07-30 18:45:03
1,2000-07-30 18:20:47
2,2000-07-30 18:37:04
3,2000-07-30 19:03:35
4,2000-07-30 18:48:51


## Функция concat()
Необходимо объединить таблицы ratings1 & ratings2, для этого воспользуемся встроенной функцией concat(), которая позволяет склеивать (конкатенировать) таблицы по строкам и столбцам
## Основные параметры функции:
 - objs - список объектов DataFrame, которые должны быть сконкатенированы
 - axis - ось конкатенирования, 0 - по строкам, 1 - по столбцам
 - join - либо inner(пересечение), либо outer (объединение)
 - ignore_index - дефолт - False, оставляет исходные индексы, True - назначает в новом порядке
 
Для корректной конкатенации по строкам объединяемые таблицы должны иметь одинаковую структуру — идентичное число и имена столбцов.

Итак, давайте склеим  ratings1 и ratings2 по строкам, так как они имеют одинаковую структуру столбцов. Для этого передадим их списком в функцию concat(). Помним, что параметр axis по умолчанию равен 0, объединение происходит по строкам, поэтому не трогаем его. 

Примечание. Обратите внимание, что concat является функцией библиотеки, а не методом DataFrame. Поэтому её вызов осуществляется как pd.concat(...).

In [4]:
ratings = pd.concat(
    [rating1, rating2],
    ignore_index=True
)
display(ratings)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100832,610,166534,4.0
100833,610,168248,5.0
100834,610,168250,5.0
100835,610,168252,5.0


In [5]:
print('Число строк ratings : ', ratings.shape[0])
print('Число строк в таблице dates: ', dates.shape[0])
print(ratings.shape[0] == dates.shape[0])

Число строк ratings :  100837
Число строк в таблице dates:  100836
False


На самом деле очень просто: при выгрузке данных информация об оценках какого-то  пользователя попала в обе таблицы (ratings1 и ratings2). В результате конкатенации случилось дублирование строк. В данном примере их легко найти — выведем последнюю строку таблицы ratings1 и первую строку таблицы ratings2:

In [6]:
display(rating2.head(1))
display(rating1.tail(1))

,userId,movieId,rating
0,274,5621,2.0


,userId,movieId,rating
40000,274,5621,2.0


Чтобы очистить таблицу от дублей, мы можем воспользоваться методом DataFrame drop_duplicates(), который удаляет повторяющиеся строки в таблице. Не забываем обновить индексы после удаления дублей, выставив параметр ignore_index в методе drop_duplicates() на значение True:

In [7]:
ratings = ratings.drop_duplicates(ignore_index=True)
ratings.shape[0]

100836

In [8]:
# теперь можно конкатенировать рейтинги с датами их выставления, для этого склеим таблицы ratings & dates по столбцам
rating_dates = pd.concat(
    [ratings, dates],
    axis=1
)
display(rating_dates.tail(7))

,userId,movieId,rating,date
100829,610,164179,5.0,2017-05-03 21:07:11
100830,610,166528,4.0,2017-05-04 06:29:25
100831,610,166534,4.0,2017-05-03 21:53:22
100832,610,168248,5.0,2017-05-03 22:21:31
100833,610,168250,5.0,2017-05-08 19:50:47
100834,610,168252,5.0,2017-05-03 21:19:12
100835,610,170875,3.0,2017-05-03 21:20:15


## Типы объединений
Есть 2 типа объединения таблиц:
 - inner (внутреннее) - при использовании такого типа, в результате объединения, остаются только те признаки, которые есть в обеих объединяемых таблицах, строки, для которых не было найдено совпадений - удаляются
 - outer (внешнее) - данный тип делится на подтипы:
    - full - используется по умолчанию и объединяет все варианты в обеих таблицах
    - left - для всех записей из левой таблицы (ratings), ведется поиск соответствий в правой таблице (movies). В результатирующей таблице останутся только те значения, которым были найдены соответствия
    - right - аналогично left

## Метод объединения join
Для объединения двух таблиц по индексам используется метод DataFrame join(). Однако данный метод можно применить и для того, чтобы объединить таблицы по ключевому столбцу

Основные параметры методы join():
 - other - таблица, которую мы присоединяем, при объединении она является "правой", а исходная таблица, от имени которой вызывается метод, является "левой"
 - how - параметр типа объединения, может принимать значения: inner, left(outer), right(outer) & outer. Дефолт параметр - left
 - on - параметр, который определяет по какому столбцу в "левой" таблице происходит объединение по индексам из правой
 -lsuffix & rsuffix - дополнения(суффиксы) к названиям одноименных столбцов в левой и правой таблицах

In [12]:
joined_false = rating_dates.join(
    movies,
    rsuffix='_left',
    how='right'
)
display(joined_false)

,userId,movieId,rating,date,movieId_left,title,genres
0,1,1,4.0,2000-07-30 18:45:03,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,2,Jumanji (1995),Adventure|Children|Fantasy
2,1,6,4.0,2000-07-30 18:37:04,3,Grumpier Old Men (1995),Comedy|Romance
3,1,47,5.0,2000-07-30 19:03:35,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,1,50,5.0,2000-07-30 18:48:51,5,Father of the Bride Part II (1995),Comedy
...,...,...,...,...,...,...,...
9737,64,3481,4.0,2006-10-22 12:37:45,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,64,3489,3.0,2006-10-22 23:28:09,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,64,3499,4.5,2006-10-22 23:26:41,193585,Flint (2017),Drama
9740,64,3510,3.0,2006-10-22 23:27:26,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


Однако это не тот результат, который мы хотели, ведь мы не получили соответствия фильмов и их рейтингов. Чтобы совместить таблицы по ключевому столбцу с помощью метода join(), необходимо использовать ключевой столбец в «правой» таблице в качестве индекса. Это можно сделать с помощью метода set_index(). Также необходимо указать название ключа в параметре on.

In [14]:
joined = rating_dates.join(
    movies.set_index('movieId'),
    on='movieId',
    how='left'
)
display(joined)

,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
...,...,...,...,...,...,...
100831,610,166534,4.0,2017-05-03 21:53:22,Split (2017),Drama|Horror|Thriller
100832,610,168248,5.0,2017-05-03 22:21:31,John Wick: Chapter Two (2017),Action|Crime|Thriller
100833,610,168250,5.0,2017-05-08 19:50:47,Get Out (2017),Horror
100834,610,168252,5.0,2017-05-03 21:19:12,Logan (2017),Action|Sci-Fi


## Метод объединения merge()
Аналогично предыдущему, метод merge() предназначен для слияния двух таблиц по ключевым столбцам или индексам. Однако в отличие от join(), merge() предлагает более гибкий способ объединения, благодаря чему более популярен

Основные параметры метода merge():
 - right - присоединяемая таблица, по дефолту "правая"
 - how - параметр типа объединения, дефолт - inner
 - on - параметр, который определяет по какому столбцу происходит объединение, определяется автоматически, но лучше выставить вручную
 - lefn_on - если названия столбцов в левой и правой таблицах не совпадают, то данный параметр отвечает за наименования ключевого столбца исходной таблицы
 - right_on - аналогично предыдущему, отвечает за наименование ключевого столбца присоединяемой таблицы

In [15]:
merged = rating_dates.merge(
    movies,
    on= 'movieId',
    how='left'
)
display(merged.head())

,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [16]:
# Проверим, что число строк в результатирующей таблице, совпадает с rating_dates

print(rating_dates.shape[0])
print(merged.shape[0])
print(rating_dates.shape[0] == merged.shape[0])

100836
100836
True


In [18]:
#Найти ответ нам поможет пример. Объединим ratings_dates с movies по ключевому столбцу movieId, но с параметром how='outer' 
# (full outer) и выведем размер таблицы, а также её «хвост»:
merged2 = rating_dates.merge(
    movies,
    on='movieId',
    how='outer'
)
print(merged2.shape[0])
merged2.tail()

100854


,userId,movieId,rating,date,title,genres
100849,184.0,193581,4.0,2018-09-16 14:44:42,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
100850,184.0,193583,3.5,2018-09-16 14:52:25,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
100851,184.0,193585,3.5,2018-09-16 14:56:45,Flint (2017),Drama
100852,184.0,193587,3.5,2018-09-16 15:00:21,Bungo Stray Dogs: Dead Apple (2018),Action|Animation
100853,331.0,193609,4.0,2018-09-17 04:13:26,Andrew Dice Clay: Dice Rules (1991),Comedy


In [19]:
merge_ratings = rating1.merge(rating2, how='outer')
print(merge_ratings.shape[0])
display(merge_ratings)

100836


,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


Обратите внимание, что при использовании метода merge() для склейки двух таблиц у нас автоматически пропали дубликаты, которые мы видели при использовании метода concat(). Это особенность метода merge() — автоматическое удаление дублей.

In [28]:
a = pd.DataFrame({'A': ['a', 'b', 'c'], 'B': [103, 214, 124], 'C': [1, 4, 2]})
b = pd.DataFrame({'V': ['d', 'b', 'c'], 'U': [1393.7, 9382.2, 1904.5], 'C': [1, 3, 2]})
a.merge(b, how='right', on='C')

,A,B,C,V,U
0,a,103.0,1,d,1393.7
1,NaN,NaN,3,b,9382.2
2,c,124.0,2,c,1904.5


In [37]:
items_df = pd.DataFrame({
            'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 100132, 312394],
            'vendor': ['Samsung', 'LG', 'Apple', 'Apple', 'LG', 'Apple', 'Samsung', 'Samsung', 'LG', 'ZTE'],
            'stock_count': [54, 33, 122, 18, 102, 43, 77, 143, 60, 19]
        })

purchase_df = pd.DataFrame({
            'purchase_id': [101, 101, 101, 112, 121, 145, 145, 145, 145, 221],
            'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 103845, 100132],
            'price': [13900, 5330, 38200, 49990, 9890, 33000, 67500, 34500, 89900, 11400]
        })
merged3 = items_df.merge(
    purchase_df,
    how='right',
    on='item_id'
)
income = merged3['price'].sum() * merged3['stock_count']
print(income)

0    19094940.0
1    11669130.0
2    43140420.0
3     6364980.0
4    36068220.0
5    15205230.0
6    27227970.0
7    50566230.0
8           NaN
9    21216600.0
Name: stock_count, dtype: float64
